# Exemplo ventanas deslizantes
En primeiro lugar iniciamos a sesión como nos casos anteriores:


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
import string

spark = SparkSession.builder \
    .appName("xanelas-2") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()

print("Versión: ",spark.version)



:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hadoop/.ivy2/cache
The jars for the packages stored in: /home/hadoop/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5f36cb50-7d31-417d-943c-1c876515402f;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.7 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.fi

Versión:  3.5.7


Neste exemplo imos ler, de novo, os datos desde un *socket*. Antes de nada póñese en marcha co seguinte comando:
```bash
nc -lk 9999
```

O seguinte paso é poñer en marcha o *stream* de lectura, exactamente igual ca no caso anterior.

In [2]:
df_lineas = spark.readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", "9999") \
    .option('includeTimestamp', 'true')\
    .load()

df_lineas.printSchema()

root
 |-- value: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)



A creación de *df_palabras* tamén é exactamente igual ao caso anterior.

In [3]:
from pyspark.sql.functions import explode, split
df_palabras = df_lineas.select(
    explode(split(df_lineas.value, ' ')).alias('palabra'),
    df_lineas.timestamp)

Como neste caso imos usar **xanelas deslizantes**, ademais de indicar a duración da xanela (2 minutos), indicamos tamén o **desprazamento** (1 minuto).


In [4]:
from pyspark.sql.functions import window
windowed_counts = df_palabras.groupBy(
    window(df_palabras.timestamp, "2 minutes", "1 minute"), df_palabras.palabra
).count().orderBy('window')

Finalmente, lanzamos a consulta indicando a saída pola consola. Como poderedes observar, cando introducimos unha nova liña as súas palabras engádense a dúas xanelas: a do minuto actual e a do anterior.


In [5]:
query = windowed_counts \
          .writeStream \
          .outputMode("complete") \
          .format("console") \
          .queryName("consulta1") \
          .option("truncate","false") \
          .start()

-------------------------------------------
Batch: 0
-------------------------------------------
+------+-------+-----+
|window|palabra|count|
+------+-------+-----+
+------+-------+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+--------+-----+
|window                                    |palabra |count|
+------------------------------------------+--------+-----+
|{2026-02-10 10:52:00, 2026-02-10 10:54:00}|ola     |1    |
|{2026-02-10 10:52:00, 2026-02-10 10:54:00}|caracola|1    |
|{2026-02-10 10:53:00, 2026-02-10 10:55:00}|ola     |1    |
|{2026-02-10 10:53:00, 2026-02-10 10:55:00}|caracola|1    |
+------------------------------------------+--------+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+------------------------------------------+--------+-----+
|window                                    |palabra |count|
+------------------------------------------+--------+-----+
|{2026-02-10 10:52:00, 2026-02-10 10:54:00}|ola     |1    |
|{2026-02-10 10:52:00, 2026-02-10 10:54:00}|que     |1    |
|{2026-02-10 10:52:00, 2026-02-10 10:54:00}|tal     |1    |
|{2026-02-10 10:52:00, 2026-02-10 10:54:00}|estás   |1    |
|{2026-02-10 10:52:00, 2026-02-10 10:54:00}|caracola|1    |
|{2026-02-10 10:53:00, 2026-02-10 10:55:00}|que     |1    |
|{2026-02-10 10:53:00, 2026-02-10 10:55:00}|ola     |1    |
|{2026-02-10 10:53:00, 2026-02-10 10:55:00}|tal     |1    |
|{2026-02-10 10:53:00, 2026-02-10 10:55:00}|estás   |1    |
|{2026-02-10 10:53:00, 2026-02-10 10:55:00}|caracola|1    |
+------------------------------------------+--------+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+------------------------------------------+--------+-----+
|window                                    |palabra |count|
+------------------------------------------+--------+-----+
|{2026-02-10 10:52:00, 2026-02-10 10:54:00}|ola     |1    |
|{2026-02-10 10:52:00, 2026-02-10 10:54:00}|que     |2    |
|{2026-02-10 10:52:00, 2026-02-10 10:54:00}|tal     |1    |
|{2026-02-10 10:52:00, 2026-02-10 10:54:00}|estás   |1    |
|{2026-02-10 10:52:00, 2026-02-10 10:54:00}|dices   |1    |
|{2026-02-10 10:52:00, 2026-02-10 10:54:00}|caracola|1    |
|{2026-02-10 10:52:00, 2026-02-10 10:54:00}|me      |1    |
|{2026-02-10 10:53:00, 2026-02-10 10:55:00}|que     |2    |
|{2026-02-10 10:53:00, 2026-02-10 10:55:00}|ola     |1    |
|{2026-02-10 10:53:00, 2026-02-10 10:55:00}|dices   |1    |
|{2026-02-10 10:53:00, 2026-02-10 10:55:00}|tal     |1    |
|{2026-02-10 10:53:00, 2026-02-10 10:55:00}|me      |1    |
|{2